<a href="https://colab.research.google.com/github/dannynacker/strobe_entrainment_periodicity_MSc/blob/main/aperiodic_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

def poisson_flash_times(frequency, duration, duty_cycle, min_interval=0.025, max_interval_factor=3.0, max_freq=200.0):
    """Generates Poisson-distributed flash onset times ensuring correct flash count and duty cycle adjustments."""
    expected_flashes = int(round(duration * frequency))
    cycle_length = 1.0 / max(frequency, 0.1)
    times = []
    time = 0

    while len(times) < expected_flashes and time < duration:
        interval = np.random.exponential(cycle_length * 0.8)
        interval = min(interval, cycle_length * max_interval_factor)
        interval = max(interval, min_interval)

        on_duration = interval * (duty_cycle / 100)
        time += interval
        if time < duration:
            times.append((time, on_duration))

    return times

def regularise_strobe(signal, min_interval=0.025):
    """Regularizes strobe sequence by handling overlaps but maintaining frequency integrity."""
    output_signal = []
    last_flash_end = -np.inf
    for onset, duration in signal:
        if onset >= last_flash_end + min_interval:
            output_signal.append((onset, duration))
            last_flash_end = onset + duration
    return output_signal

def calculate_effective_frequency(signal, duration):
    """Computes the effective frequency based on the number of flashes in the signal."""
    return len(signal) / duration

def optimize_initial_frequency(target_Fe, duration, duty_cycle, tolerance=0.01, max_iterations=10000, max_freq=200.0):
    """Finds the optimal initial frequency that produces the desired effective frequency, clamping within range."""
    F = min(target_Fe * 1.2, max_freq)
    iteration = 0

    while iteration < max_iterations:
        flash_times = poisson_flash_times(F, duration, duty_cycle, max_freq=max_freq)
        flash_times = regularise_strobe(flash_times)
        Fe = calculate_effective_frequency(flash_times, duration)

        if abs(Fe - target_Fe) <= tolerance:
            return min(F, max_freq), Fe, flash_times

        F = min(max(0.1, F + 0.15 if Fe < target_Fe else F - 0.15), max_freq)
        iteration += 1

    print("Warning: Max iterations reached without finding precise Fe match.")
    return F, Fe, flash_times

def generate_strobe_sequence():
    """Interactive function to generate a strobe sequence file, ensuring correct flash count."""
    stimulation = input("Choose stimulation type (Periodic/Aperiodic): ").strip()
    duration = float(input("Enter total sequence duration (seconds): "))
    num_steps = int(input("Enter number of steps: "))

    session_params = []
    for i in range(num_steps):
        print(f"Step {i+1}:")
        step_duration = float(input("Enter step duration (seconds): "))
        start_Fe = float(input("Enter start effective frequency (Hz): "))
        end_Fe = float(input("Enter end effective frequency (Hz): "))
        start_d = float(input("Enter start duty cycle (0-1): "))
        end_d = float(input("Enter end duty cycle (0-1): "))
        start_l = int(input("Enter start brightness (0-100): "))
        end_l = int(input("Enter end brightness (0-100): "))
        wave_type = input("Enter wave type (Square/Sine): ").strip()
        led_config = [int(input(f"LED Set {i+1} (1=On, 0=Off): ")) for i in range(4)]

        session_params.append({
            "duration": step_duration,
            "frequency": (start_Fe, end_Fe),
            "duty_cycle": (start_d, end_d),
            "brightness": (start_l, end_l),
            "wave_type": wave_type,
            "led_config": led_config
        })

    generate_session("strobe_generated.txt", stimulation, session_params)
    print("Generated strobe_generated.txt")

def generate_session(file_name, stimulation_type, session_params):
    """Generates a full session file based on predefined parameters."""
   # Calculate total duration correctly
    total_duration = round(sum(d["duration"] for d in session_params), 1)
    minutes = int(total_duration // 60)
    seconds = total_duration % 60

    # Ensure the TIM format correctly represents minutes and seconds
    tim_string = f'TIM"00:{minutes:02}:{seconds:04.1f}"'
    dur_string = f'DUR"{total_duration:.1f}"'
    min_interval = 0.025
    max_freq = 200.0

    for params in session_params:
        start_Fe, end_Fe = params["frequency"]
        start_d, end_d = int(params["duty_cycle"][0] * 100), int(params["duty_cycle"][1] * 100)
        start_l, end_l = params["brightness"]
        wave_type = 1 if params["wave_type"] == "Square" else 2
        led_config = params["led_config"]
        duration = params["duration"]

        if stimulation_type == "Aperiodic":
            F, Fe, flash_times = optimize_initial_frequency(start_Fe, duration, start_d, max_freq=max_freq)
        else:
            Fe, flash_times = start_Fe, [(j * (1.0 / start_Fe), (start_d / 100) * (1.0 / start_Fe)) for j in range(int(duration * start_Fe))]
            flash_times = regularise_strobe(flash_times)

        previous_onset = 0
        for onset, on_duration in flash_times:
            step_duration = max(onset - previous_onset, min_interval)
            previous_onset = onset + on_duration
            step_freq = Fe

            step_string = f'STP"{step_duration:.3f},{wave_type},{start_Fe:.2f},{end_Fe:.2f},{start_d},{end_d},{led_config[0]},{led_config[1]},{led_config[2]},{led_config[3]},{start_l},{end_l},' * 4
            output.append(step_string.strip(','))

    with open(file_name, "w") as f:
        f.write('\n'.join(output))
    print(f"Generated {file_name}")

# Run the generator
generate_strobe_sequence()